In [90]:
import GEOparse
import sys
import pandas 
pandas.set_option('display.max_colwidth', -1)
import re

In [217]:
#filepath = "_data/genomic_data/melanoma/GSE8401_family.soft.gz" 
#filepath = "_data/genomic_data/melanoma_2/GSE7553_family.soft.gz"
filepath = "_data/genomic_data/melanoma_2/GSE15605_family.soft.gz"

In [218]:
dataSet = GEOparse.get_GEO(filepath=filepath)

21-May-2018 21:33:49 INFO GEOparse - Parsing _data/genomic_data/melanoma_2/GSE15605_family.soft.gz: 
21-May-2018 21:33:49 DEBUG GEOparse - DATABASE: GeoMiame
21-May-2018 21:33:49 DEBUG GEOparse - SERIES: GSE15605
21-May-2018 21:33:49 DEBUG GEOparse - PLATFORM: GPL570
/usr/local/lib/python3.6/dist-packages/GEOparse/GEOparse.py:332: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  return DataFrame.from_csv(StringIO(data), index_col=None, sep="\t")
/usr/local/lib/python3.6/dist-packages/GEOparse/GEOparse.py:521: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  gpls[entry_name] = parse_GPL(data_group, entry_name)
21-May-2018 21:33:51 DEBUG GEOparse - SAMPLE: GSM390208
21-May-2018 21:33:51 DEBUG GEOparse - SAMPLE: GSM390209
21-May-2018 21:33:51 DEBUG GEOparse - SAMPLE: GSM3902

In [219]:
dataSet.database.metadata

{'name': ['Gene Expression Omnibus (GEO)'],
 'institute': ['NCBI NLM NIH'],
 'web_link': ['http://www.ncbi.nlm.nih.gov/geo'],
 'email': ['geo@ncbi.nlm.nih.gov']}

In [220]:
print("Dataset name:{} ".format(dataSet.name))
print("+"*50)
print(dataSet.show_metadata())

Dataset name:GSE15605 
++++++++++++++++++++++++++++++++++++++++++++++++++
!Series_title = Transcriptome profiling identifies HMGA2 as a novel gene in melanoma progression
!Series_geo_accession = GSE15605
!Series_status = Public on Jun 08 2012
!Series_submission_date = Apr 08 2009
!Series_last_update_date = Apr 23 2018
!Series_pubmed_id = 23633021
!Series_summary = The identification of novel tumor-specific markers may improve understanding of melanoma progression and prognostic accuracy. Whole genome expression profiling of 46 primary melanomas, 12 metastases, and 16 normal skin samples using Affymetrix U133 PLUS 2.0 array generated gene lists including both known and new melanoma genes.
!Series_summary = The molecular genetic alterations contributing to the pathogenesis of melanoma are incompletely defined and few independent prognostic features have been identified beyond the pathologic characteristics of the primary tumor. Early stage melanoma is frequently curable, in contrast to t

In [221]:
genome_table = dataSet.pivot_samples('VALUE')
genome_df = genome_table.T

In [225]:
dataSet.gpls[list(dataSet.gpls.keys())[0]].columns

,description
ID,"Affymetrix Probe Set ID LINK_PRE:""https://www.affymetrix.com/LinkServlet?array=U133PLUS&probeset="""
GB_ACC,"GenBank Accession Number LINK_PRE:""http://www.ncbi.nlm.nih.gov/nuccore/?term="""
SPOT_ID,identifies controls
Species Scientific Name,The genus and species of the organism represented by the probe set.
Annotation Date,The date that the annotations for this probe array were last updated. It will generally be earlier than the date when the annotations were posted on the Affymetrix web site.
Sequence Type,
Sequence Source,The database from which the sequence used to design this probe set was taken.
Target Description,
Representative Public ID,"The accession number of a representative sequence. Note that for consensus-based probe sets, the representative sequence is only one of several sequences (sequence sub-clusters) used to build the consensus sequence and it is not directly used to derive the probe sequences. The representative sequence is chosen during array design as a sequence that is best associated with the transcribed region being interrogated by the probe set. Refer to the ""Sequence Source"" field to determine the database used."
Gene Title,Title of Gene represented by the probe set.


In [226]:
characteristics =[col_name for col_name in dataSet.phenotype_data.columns 
                  if any(target in col_name for target in ['characteristics',
                                                           'label', 
                                                           'type', 
                                                           'source_name',
                                                           'organisme',
                                                           'molecule',
                                                           'title'])]

In [227]:
for char in characteristics:
    if dataSet.phenotype_data[char].unique().shape[0] > 1:
        print(char,':',dataSet.phenotype_data[char].unique().shape,':', dataSet.phenotype_data[char].unique())

characteristics_ch1.0.gender : (2,) : ['m' 'f']
characteristics_ch1.1.age at diagnosis : (44,) : ['79' '47' '44' '85' '39' '41' '50' '74' '56' '73' '58' '20' '62' '55'
 '52' '66' '60' '76' '59' '63' '42' '81' '83' '80' '72' '61' '33' '45'
 '51' '70' '67' '43' '48' '36' '87' '92' '64' '40' '27' '71' '77' '90'
 '53' '89']
characteristics_ch1.2.diagnosis : (13,) : ['NORMAL SKIN' 'DESMOPLASTIC MELANOMA' 'SSM' 'NODULAR MELANOMA' 'MIS'
 'SPINDLE CELL MELANOMA' 'ACRAL MELANOMA' 'DERMAL  MELANOMA'
 'POLYPOID MELANOMA' 'LMM' 'REGIONAL LYMPH NODE METASTASIS'
 'DISTANT METASTASIS' 'SUBCUTANIOUS SOFT TISSUE METASTASIS']
characteristics_ch1.3.braf mutation status : (4,) : [nan 'V600R' 'WT' 'V600K']
characteristics_ch1.3.distant metastases : (2,) : [nan 'YES']
characteristics_ch1.3.regional metastases : (4,) : ['not applicable' 'NO' 'YES' nan]
characteristics_ch1.4.braf mutation status : (4,) : [nan 'V600E' 'WT' 'V600K']
characteristics_ch1.4.distant metastases : (4,) : ['not applicable' 'NO' 'YES' 

In [83]:
target_variable = 'source_name_ch1'

In [169]:
test = re.compile(r'([0-9]{4}\_[0-9]{2})')
test.sub('', 'basal cell carcinoma 7845_17')

'basal cell carcinoma '

In [212]:
filters = [re.compile(r'(group [0-9])|\,'), 
           re.compile(r'([0-9]{4}\_[0-9]{1,2})'),
           re.compile(r'([0-9]{4}[a-z]?\-[0-9]{1,2})'),
           re.compile(r'([0-9]{4}\s[a-z][0-9]?)'),
           re.compile(r'([a-z][0-9]{4}[a-z])'),
           re.compile(r'[0-9]'),
           re.compile(r'(^\s)|(\s$)')
          ]

def filter_string(x):
    for re_filter in filters:
        x = re_filter.sub('', x.lower())
    return x
df_type = dataSet.phenotype_data[['source_name_ch1', 'title']].applymap(lambda x: filter_string(x))

In [192]:
#list_type = []
#for idx, row in dataSet.phenotype_data.iterrows():
#    list_type.append({"sample":idx, "type":row[target_variable]})
#df_type = pandas.DataFrame(list_type)

In [214]:
final_df = genome_df.merge(df_type, how='inner', left_index=True, right_index=True)